In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
from env import get_db_url
import split_scale as ss
import features as fe
import modeling as mod
import evaluate as ev
import warnings
from statsmodels.formula.api import ols
warnings.filterwarnings("ignore")


# Building the Minimally Viable Product


## Acquiring the data

- Make query from Zillow Database, for the MVP we'll use only the Bed and Bathroom counts and the Squarefootage to predict home value

In [2]:
query = '''
select bathroomcnt as bathrooms, bedroomcnt as bedrooms, calculatedfinishedsquarefeet as squarefeet, taxvaluedollarcnt as home_value
from properties_2017
join predictions_2017 as pred
using(parcelid)
where pred.`transactiondate` like '%%-05-%%' or pred.`transactiondate` like '%%-06-%%'
'''
df = pd.read_sql(query, get_db_url('zillow'))

In [7]:
df.head()

,bathrooms,bedrooms,squarefeet,home_value
0,3.0,3.0,1534.0,43439.0
1,2.0,3.0,1458.0,136104.0
2,1.0,2.0,1421.0,35606.0
3,3.0,4.0,2541.0,880456.0
4,2.0,3.0,1650.0,614000.0


## Cleaning the data

- All columns are floats: good
- squarefeet column contains nulls: gotta fix that
- features are in different units: lets give those a standard scale for now

In [5]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21944 entries, 0 to 21943
Data columns (total 4 columns):
bathrooms     21938 non-null float64
bedrooms      21938 non-null float64
squarefeet    21883 non-null float64
home_value    21938 non-null float64
dtypes: float64(4)
memory usage: 685.8 KB


,bathrooms,bedrooms,squarefeet,home_value
count,21938.000000,21938.000000,21883.000000,2.193800e+04
mean,2.319924,3.065685,1808.392268,5.034094e+05
std,0.995531,1.125493,973.433723,6.450342e+05
min,0.000000,0.000000,242.000000,1.050400e+04
25%,2.000000,2.000000,1194.000000,2.128388e+05
50%,2.000000,3.000000,1562.000000,3.682350e+05
75%,3.000000,4.000000,2145.000000,5.888448e+05
max,11.000000,12.000000,35640.000000,2.385837e+07


In [6]:
df = df.dropna()

## Prepare data

- need to split data and check if transformation is necessary
- features are in different units: lets give those a standard scale for now

In [9]:
train, test = ss.split_my_data(df)
scaler, scaled_train, scaled_test = ss.standard_scaler(train, test)

In [13]:
scaled_train.head()

,bathrooms,bedrooms,squarefeet,home_value
13021,2.697703,2.624085,1.857324,1.046089
14124,-0.331254,-0.068917,-0.281756,-0.613482
1156,-0.331254,-0.966585,-0.307310,-0.632764
21561,-1.340907,-1.864252,-0.645899,0.879715
18089,-1.340907,-0.966585,-0.939770,-0.610689


## Feature Engineering

- first we'll establish our features
- then we'll delve into how strong they are

In [86]:
features = ['bathrooms', 'bedrooms', 'squarefeet']
target = ['home_value']

In [20]:
fe.lasso_cv_coef(scaled_train[features], scaled_train[target])

bathrooms     0.123688
bedrooms     -0.230038
squarefeet    0.638326
dtype: float64

In [23]:
fe.optimum_feature_count(scaled_train[features], scaled_train[target], scaled_test[features], scaled_test[target])

3

## Modeling

- generally modeling is preemptave at this point, but we're just making a quick, basic model right now and will come back to it later as we dive deeper into the data

In [93]:
regr = ols('home_value ~ bathrooms + bedrooms + squarefeet', data= scaled_train).fit()
scaled_train['yhat'] = regr.predict(scaled_train)

In [95]:
#find r2
ev.model_significance(scaled_train[target], scaled_train['yhat'], regr)

{'exp_var': 0.37693102690329505, 'f_pval': 0.0}